In [1]:
import torch
import transformers

transformers.set_seed(42)
DEVICE = "cuda:7"
NUM_SAMPLES = 32
CHECKER_TOKENIZER = transformers.AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
CHECKER_MODEL = transformers.AutoModelForCausalLM.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")

/home/tutugarin/fairseq_virtualenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-03 20:32:03.809205: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 20:32:04.988794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [00:40<00:00, 13.41s/it]


In [2]:
# gpt2_generator = transformers.pipeline('text-generation', model='gpt2-xl', device=DEVICE)

In [3]:
# gpt2_texts = []
# for _ in range(NUM_SAMPLES):
#     text = gpt2_generator("Once upon a time", max_length=100, num_return_sequences=1)[0]["generated_text"]
#     gpt2_texts.append(text)
# gpt2_texts = [text.replace("\n", "") for text in gpt2_texts]
# with open("/home/tutugarin/gpt2_tine_stories/gpt2_texts.txt", 'w') as file:
#     for text in gpt2_texts:
#         print(text, file=file)

In [4]:
with open("/home/tutugarin/gpt2_tine_stories/gpt2_texts.txt") as file:
    gpt2_texts = file.readlines()
gpt2_texts[0]

'Once upon a time, the name "the Black Lives Matter movement" would have seemed so alien to the American imagination that the phrase would have been regarded as completely innocuous, or even humorous. But in the past decade, an organized effort from Black youth has made the movement a lightning rod for widespread protests around the country from Ferguson, Mo. to Baltimore, Md. — and one that even Republican presidential candidates have picked up on.( Also on POLITICO: Who\'s talking about Black lives?\n'

In [19]:
with open("/home/tutugarin/gpt2_tine_stories/gen_3.txt") as file:
    ershov_texts = file.readlines()
ershov_texts[0]

'Once upon a time there was an old man. his name had a big box of junk in the corner. every day he would take out the box of old things and put them in the corner of his house to make something new and exciting. one morning he was feeling a bit tired so he decided he wanted a nap. so he took a nap and when he woke it was all done, he opened the box and took out all of the old junk. it was so soft, it was like it had been in the box for a very long, curled up in his arms and closed its eyes tight. the man was so tired he fell asleep right there in the box. when he woke up, he was so surprised to find a little puppy inside the box. he smiled and said "thank you for making me a nap, little pup. i\'ve been so sleepy and i was so tired!"\n'

In [20]:
def ppl(model_texts, checker_model=CHECKER_MODEL, checker_tokenizer=CHECKER_TOKENIZER):
    checker_model.to(DEVICE)
    encodings = checker_tokenizer("\n\n".join(model_texts), return_tensors="pt")

    max_length = 64
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(DEVICE)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = checker_model(input_ids, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over input tokens.
            # Multiply it with trg_len to get the summation instead of average.
            # We will take average over all the tokens to get the true average
            # in the last step of this example.
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    return torch.exp(torch.stack(nlls).sum() / end_loc).item()

In [21]:
ppl(gpt2_texts)

22.08954620361328

In [22]:
ppl(ershov_texts)

21.377511978149414